In [2]:
import os
import json
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI
load_dotenv()

MODEL = 'gpt-4o-mini'
openai = OpenAI()

system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [3]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for human, assistant in history:
        messages.append({"role": "user", "content": human})
        messages.append({"role": "assistant", "content": assistant})

    messages.append({"role": "user", "content": message})
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat).launch()

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [4]:
ticket_prices = {
    "london": 100,
    "paris": 150,
    "new york": 300,
    "tokyo": 500,
    "berlin": 200,
}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called with destination_city: {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")



get_ticket_price("Berlin")

Tool get_ticket_price called with destination_city: Berlin


200

In [5]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticker price, for exampl when a customer asls: 'How much is a ticket to this city?'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to."
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }   
}

In [6]:
tools = [{'type': 'function', 'function': price_function}]


def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    print(tool_call)
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get("destination_city")
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city, "price": price}),
        "tool_call_id": message.tool_calls[0].id
    }
    return response, city


def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for human, assistant in history:
        messages.append({"role": "user", "content": human})
        messages.append({"role": "assistant", "content": assistant})

    messages.append({"role": "user", "content": message})
    
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools
    )

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    

    


    return response.choices[0].message.content


gr.ChatInterface(fn=chat).launch()

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Handling tool calls: [ChatCompletionMessageToolCall(id='call_R8tTc75PyDlGh53BT8Ge7vdp', function=Function(arguments='{"destination_city":"London"}', name='get_ticket_price'), type='function')]
Tool get_ticket_price called with destination_city: London
Handling tool calls: [ChatCompletionMessageToolCall(id='call_7dHbt0sv3LTnSuAZO8jazGlN', function=Function(arguments='{"destination_city":"Paris"}', name='get_ticket_price'), type='function')]
Tool get_ticket_price called with destination_city: Paris
Handling tool calls: [ChatCompletionMessageToolCall(id='call_t5bMOMr9XPDYe75WSOeni91n', function=Function(arguments='{"destination_city": "London"}', name='book_ticket'), type='function'), ChatCompletionMessageToolCall(id='call_2Mi1nM2GVLAktZgFUULkrOkw', function=Function(arguments='{"destination_city": "Paris"}', name='book_ticket'), type='function')]
Tool book_ticket called with destination_city: London
Tool book_ticket called with destination_city: Paris


In [7]:

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called with destination_city: {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")


def book_ticket(destination_city):
    print(f"Tool book_ticket called with destination_city: {destination_city}")
    city = destination_city.lower()
    if city in ticket_prices:
        return f"Ticket to {city.title()} booked successfully!"
    else:
        return "Booking failed. Unknown destination city."





book_function = {
    "name": "book_ticket",
    "description": "Book a ticket to the destination city. Call this function whenever you are asked to book a ticket.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to."
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}


tools = [{'type': 'function', 'function': price_function},
         {'type': 'function', 'function': book_function}]


def handle_tool_calls(message):
    print(f"Handling tool calls: {message.tool_calls}")
    responses = []
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get("destination_city")
        if tool_call.function.name == "book_ticket":
            res = book_ticket(city)
        elif tool_call.function.name == "get_ticket_price":
            res = get_ticket_price(city)

        
        response = {
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps({
                "destination_city": city,
                "res": res
            })
        }

        responses.append(response)
    return responses



def chat(message, history):
    messages = [{'role': 'system', 'content': system_message}]
    for human, assistant in history:
        messages.append({'role': 'user', 'content': human})
        messages.append({'role': 'assistant', 'content': assistant})
    
    messages.append({'role': 'user', 'content': message})

    response = openai.chat.completions.create(
        model = MODEL,
        messages = messages,
        tools = tools
    )
    


    if response.choices[0].finish_reason == "tool_calls":
        assistant_msg = response.choices[0].message
        tool_responses = handle_tool_calls(assistant_msg)
        messages.append({
            "role": "assistant",
            "tool_calls": assistant_msg.tool_calls,
            "content": assistant_msg.content
        })
        messages.extend(tool_responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    


    return response.choices[0].message.content



gr.ChatInterface(fn=chat).launch()

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
